# Multi-turn prompts in conversations

## 多轮对话
之前我们探讨了如何在单一对话下使用 prompt 来让模型进行输出，但是在实际应用中，我们通常需要多个对话来完成一个任务，例如，在聊天机器人中，用户可能会针对一个问题进行多次对话，以获取更详细的信息。

我们先来看看如何进行多轮对话。

In [ ]:
import os
from openai import OpenAI

from dotenv import load_dotenv
load_dotenv()

# 从环境变量中获取 API 密钥
api_key = os.getenv("DEEPSEEK_API_KEY")

llm = OpenAI(api_key=api_key, base_url="https://api.deepseek.com")

messages = []

def get_response(prompt):
    global messages
    messages.append({"role": "user", "content": prompt})
    response = llm.chat.completions.create(
        model="deepseek-chat",
        messages=messages
    )
    messages.append(response.choices[0].message)
    return 'AI: ' + response.choices[0].message.content + '\n'

# 示例多轮对话
single_turn_prompt = "什么是大语言模型？"
print(single_turn_prompt)
print(get_response(single_turn_prompt))

follow_up_prompt = "大语言模型有哪些后训练方式？"
print(follow_up_prompt)
print(get_response(follow_up_prompt))

## 使用 LangChain
可以看到我们进行多轮对话的方式就是将每一次的对话存进一个消息列表 messages 中，每次新输入都要将其加入到这个消息列表。那么有没有更方便的方法？有的，我们可以借助 LangChain 的 ConversationBufferMemory 类来实现。

在此之前，我们需要简单熟悉一下 LangChain 的基本用法，其实在之前写 prompt 模板时，我们就已经使用过 LangChain 的 PromptTemplate 类了。

于是，让我们先对此前学习过的任务进行小小的修改，以便于后续更好地使用 LangChain。

In [1]:
import os
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory

# 初始化语言模型
llm = ChatOpenAI(
    model="deepseek-chat",
    base_url="https://api.deepseek.com",
    api_key=os.getenv("DEEPSEEK_API_KEY")
)

In [ ]:
basic_prompt = "请用一句话解释一下什么是 Multi-Turn Prompts ? "

print(llm.invoke(basic_prompt).content)

In [ ]:
structured_prompt = PromptTemplate(
    input_variables=["topic"],
    template="请提供 {topic} 的定义，然后解释其重要性。"
)

chain = structured_prompt | llm
input_variables = {"topic": "Multi-Turn Prompts"}
print(chain.invoke(input_variables).content)

可以看到，我们使用了 LangChain 的 ChatOpenAI 类来初始化一个语言模型，并使用 PromptTemplate 类来定义一个模板，然后使用这个模板来生成一个提示。

现在我们已经可以用 LangChain 来替代之前的做法了，你可以尝试一下！

## 使用 Multi-turn prompts
接下来，我们可以使用 LangChain 的 ConversationChain 和 ConversationBufferMemory 来管理我们的多轮对话，并针对每轮对话提供 prompt。

In [ ]:
conversation = ConversationChain(
    llm=llm, 
    verbose=True,
    memory=ConversationBufferMemory()
)

print(conversation.predict(input="你好，我在学习关于 LLM 的知识。你能告诉我关于 LLM 的信息吗？"))
print(conversation.predict(input="最早的 LLM 是哪一个模型，是哪篇论文提出的？"))
print(conversation.predict(input="现在最新的 LLM 进行了哪些技术的优化？"))

在这里，我们将 verbose 这个参数设置为 True，你可以更好的看到 ConversationBufferMemory 发挥的作用，将每轮对话都做了存储，也就是上下文信息的存储。

实际上，我们在使用 Multi-Turn Prompts 这一技术的时候，可以写的更简洁规整一些，让输入输出看起来更像在使用一个对话机器人。

In [ ]:
prompts = [
    "请问 Prompt Engineering 属于后训练技术的一种吗？",
    "Prompt Engineering 和 Fine-tuning 有何区别？",
    "有哪些后训练技术可以不依赖 GPU 进行？"
]

conversation = ConversationChain(llm=llm, memory=ConversationBufferMemory())
for prompt in prompts:
    print(f"Q: {prompt}")
    print(f"A: {conversation.predict(input=prompt)}\n")

至此，我们完成了一个可以多轮对话的 Prompt Engineering 的探索，并且模型不仅会对每次输入的 prompt 进行输出，还会根据之前的对话记录中的 prompt 进行分析，这就是 Multi-turn prompts 技术。